## This code will iterate over multiple json files and will:
<ol>

  <li>Search</li>
  <li>Apply Orders v2 Raster Functions</li>
  <li>Order & Download</li>
  <li>Receive Order Notification (Optional)</li>

</ol>

## Orders v2 Notebook Pre-requisites:

[Planet Account](https://www.planet.com/login/?mode=signup)

[Planet API Key](https://www.planet.com/account/#/)

[Anaconda with Python 3.7](https://www.anaconda.com/download/)

   From the [Anaconda Navigator](https://docs.anaconda.com/anaconda/navigator/), open a terminal to install the geojson module: `pip install geojson` 
   
   From the [Anaconda Navigator](https://docs.anaconda.com/anaconda/navigator/), open a terminal to install the Shapely module: `conda install -c scitools shapely`
    
   Note: *In Windows, run Terminal as Administrator if receiving a write permission error

# Set Environment

In [4]:
from requests import Session, get, post
from time import sleep
import glob
import itertools
import json
import os
import geopandas as pd
import shapely.geometry
import shapely.ops
from shapely.geometry import mapping, shape


# print function to format JSON
def p(data):
    print(json.dumps(data, indent=2))

### Important References
[Data API documentation](https://developers.planet.com/docs/api/)
* [Items and Assets](https://developers.planet.com/docs/api/items-assets/)

[Orders v2 documentation](https://developers.planet.com/docs/orders/)


### Note about JSON files:

* Windows directory format for **json_path** below: `"C:\\subdirectory\\json_directory\\"`
* Mac Directory format for **json_path** below: `"/Users/username/json_directory/"`
* GeoJSON(s) can be easily generate at [GeoJSON IO](http://geojson.io)
* GeoJSON(s) must be in `.json` extension
* This Orders v2 script **only** works with GeoJSON files with a **single Polygon feature**

## Define Variables

In [5]:
#Location of json files:
json_path = '/data/ETH/' #Include trailing / in the json path
                                       #Rename to .json

#Percent coverage
coverage_thresh = 0.90

#Item Type: "PSScene4Band","SkySatScene","PSScene3Band","REScene","REOrthoTile","Sentinel2L1C","SkySatCollect","CustomL3A","PSOrthoTile","Landsat8L1G"
item_type = 'PSScene4Band'

#Order Payload Params (e.g. visual, analytic, analytic_dn, analytic_sr, etc.):
prod_bundle = "analytic_sr"

#Temporal Search Params
start_time = "2019-05-01T00:00:00.000Z"
stop_time = "2019-05-30T00:00:00.000Z"

#Other Filter Params
min_gsd = 0.0
max_cloud_cover = 0.2 #Cloud cover from 0 to 1

API_TOKEN = os.environ.get("API_TOKEN")
USER = os.environ.get("USER")

## Order Notification

In [6]:
# Choose between email notification OR webhook POST response below

# Email notification
email = True

# Webhook Notification (Paid Customers Only)
# Specify a webhook URL to be notified when the order is ready. Get a unique URL from https://webhook.site
webhook = False
webhook_url = "<your webhook url>"

## Prepare Geometry (included in free trial)

In [11]:
# Convert Shape to Pandas Dataframe
import geojson
inD = pd.read_file(os.path.join(basepath, "Ethiopia.shp"))
geojson.Feature(inD.iloc[0]['geometry'])


NameError: name 'basepath' is not defined

In [ ]:
import geojson
geojson.Feature(inD.iloc[0]['geometry'])

In [ ]:
# Clip a raster to an Area of Interest 
# The AOI can be specified by a polygon with up to 500 vertices
clip = True

## Additional Raster Functions (Paid Customers Only)

In [ ]:
# Composite a set of raster files into one output.
composite = True

# Apply arbitrary band math to produce derived raster products.
bandmath = False
band_calc = "(b4-b3)/(b4+b3)" #NDVI
pixel_type = "32R"

# Deliver TOA product (applicable for analytic product only, NOT necessary for the analytic_sr product)
toar = False
toar_scale_factor = 1000

## Initiate Session 

In [ ]:
ORDERS_URL = "https://api.planet.com/compute/ops/orders/v2/"
PLANET_API_KEY = os.getenv('PL_API_KEY') # Define PL_API_KEY as an evironment variable or type it below
#PLANET_API_KEY = "YOUR API KEY"

planet = Session()
planet.auth = (PLANET_API_KEY, "")

# STOP - NO NEED TO EDIT THE CODE BELOW

## Simplify the polygon

In [ ]:
def simplify_poly(jsondata):

    #Extract geometry from json file
    json_geom = json_data['features'][0]['geometry']
    num_coords = len(json_geom['coordinates'][0])

    print ("")
    print ("The original polygon has {} points".format(num_coords))
    print ("")
    
    #Conver to shape and simplify polygon
    shape_geom = shape(json_geom)
    simple_poly = shape_geom.simplify(0.00001, preserve_topology=True)

    #print simple_poly
    simple_geom = mapping(simple_poly)
    num_simple_coords = len(simple_geom['coordinates'][0])

    print ("")
    print ("The simplified polygon has {} points".format(num_simple_coords))
    print ("")

    json_data['features'][0]['geometry'] = simple_geom
    
    return json_data

## Exctract geometry from json file

In [ ]:
def extract_geometries(data):

    for feature in data['features']:
        newcoords =  feature['geometry']['coordinates']

    #Simplifying Polygon    

    aoi_geometry = {
      "geometry": {
          "type": "Polygon",
          "coordinates": newcoords 
      }
    }

    clip_geometry = {
          "type": "Polygon",
          "coordinates": newcoords 
      }
    
    p(aoi_geometry)
    
    return (aoi_geometry, clip_geometry)

## Build query and search functions

In [ ]:
def quick_search_all_pages(session, query):
    r = session.post('https://api.planet.com/data/v1/quick-search', 
                     json=query)
    if r.status_code != 200:
        raise Exception(r.text)
        raise Exception("HTTP %s\n\n%s\n\n%s" % (
            r.status_code, json.dumps(query, indent=2, sort_keys=True), r.text))

    data = r.json()
    items = data['features']

    while data['_links'].get('_next'):
        data = session.get(data['_links']['_next']).json()
        items += data['features']
    
    return items


def build_query(geometry, item_types=None, acquired_gte=None, acquired_lte=None,
                gsd_gte=None, cloud_cover_lte=None):
    """ example item_types: ['PSScene3Band']
                acquired_gte: "2016-01-01T00:00:00.000Z"
    """

    
    if 'features' in geometry:
        geometry = geometry['features'][0]
    
    filters = [{"type": "GeometryFilter",
                 "field_name": "geometry",
                 "config": geometry['geometry']}]
    
    if acquired_gte:
        filters.append({
            "type": "DateRangeFilter",
            "field_name": "acquired",
            "config": {"gte": acquired_gte}})

    if acquired_lte:
        filters.append({
            "type": "DateRangeFilter",
            "field_name": "acquired",
            "config": {"lte": acquired_lte}})

    if gsd_gte is not None:
        filters.append({
            "type": "RangeFilter",
            "field_name": "gsd",
            "config": {"gte": float(gsd_gte)}})
    
    if cloud_cover_lte is not None:
        filters.append({
            "type": "RangeFilter",
            "field_name": "cloud_cover",
            "config": {"lte": float(cloud_cover_lte)}})        
    
    return {
        "interval": "day",
        "item_types": item_types,
        "filter": {
            "type": "AndFilter",
            "config": filters}}

### Verify strip coverage meets coverage threshold

In [ ]:
def group_by_strip(items, geometry):
    strips = []
    aoi = shapely.geometry.Polygon(geometry['geometry']['coordinates'][0])

    for strip_id, items_iter in itertools.groupby(items, lambda item: item['properties']['strip_id']):
        strip_items = list(items_iter)
        strip_polys = []
        for item in strip_items:
            geo_type = item['geometry']['type']
            if geo_type == 'Polygon':
                coords = item['geometry']['coordinates'][0]
                strip_polys.append(shapely.geometry.Polygon(coords))
            elif geo_type == 'MultiPolygon':
                for coords in item['geometry']['coordinates'][0]:
                    strip_polys.append(shapely.geometry.Polygon(coords))
            else:
                raise Exception('type %s is unsupported !!! fixme' % geo_type)

        strip_poly = shapely.ops.cascaded_union(strip_polys)
        coverage_percent = aoi.intersection(strip_poly).area / aoi.area
        strips.append({
            'strip_id': strip_id,
            'items': strip_items,
            'coverage': coverage_percent,
            'datetime': strip_items[0]['properties']['acquired']
        })
       
    return sorted(strips, key=lambda strip: strip['datetime'], reverse=True)

### Get all the image strips

In [ ]:
def get_items_with_coverage(session, geometry, item_types, **filters):
    query_json = build_query(geometry, item_types, **filters)
    all_items = quick_search_all_pages(session, query_json)
    return group_by_strip(all_items, geometry)

### Verify Individual Images Intersect with the AOI (TM Variation in comments)

In [ ]:
#This function will verify that individual images intersect with the clip AOI to prevent order failure
def img_intersection(thresholded_strips, geometry):

    aoi = shapely.geometry.Polygon(geometry['geometry']['coordinates'][0])
    verified_all = []
    verified_items = []
    removed_items = []
    
    for i in range(0,len(thresholded_strips)):
    
        for item in thresholded_strips[i]["items"]:
            geo_type2 = item['geometry']['type']

            img_coords = shapely.geometry.Polygon(item['geometry']['coordinates'][0])
            coverage_percent = aoi.intersection(img_coords).area / aoi.area
            
            if coverage_percent > 0:
                
                strip_items = item['id']
                strip_id = item['properties']['strip_id']
                verified_items.append(item['id'])
                verified_all.append({
                    'strip_id': strip_id,
                    'items': strip_items,
                })
            
            else:
                removed_items.append(item['id'])
                continue

    return (verified_items, removed_items, verified_all)

### Group by Strips (Composite Order)

In [ ]:
def group_imgs_by_stripid(verified_all,verified):
    grouped_by_strip = []
    # group items by unique strip_id
    for key, group in itertools.groupby(verified_all, key=lambda x:x['strip_id']):
        strip_id = key
        strip_items = []
        group_values = list(group)
        for key, group in itertools.groupby(group_values, key=lambda x:x['items']):
            strip_items.append(key)

        grouped_by_strip.append({ 
            'strip_id': strip_id,
            'strip_items': strip_items
        })
    return grouped_by_strip

## Dynamically generate order_payload

In [ ]:
def generate_order_payload(dir_name,items_by_strips,item_type,prod_bundle,clip,clip_geometry,toar,toar_scale_factor,
                                  bandmath,band_calc,pixel_type,composite,email,webhook,webhook_url):
    

    composite_orders = []
    for i in range(0,len(items_by_strips)):
        
        #Grab each strip_id for formatting order_name
        strip_id = items_by_strips[i]['strip_id']
        
        #Grab all item_ids for each strip
        item_ids = items_by_strips[i]['strip_items']        
        
        #Extract Date from Image ID
        imgID = json.dumps(item_ids[0])
        date_str = imgID[1:9]

        order_name = date_str + "_" + strip_id + "_" + prod_bundle
        print (order_name)
        
        # Create a new tool dictionary
        tools = []
        
        # Add each tool component
        if clip:
            tools.append({'clip': {"aoi": clip_geometry}})
    
        if toar:
            tools.append({'toar': {"scale_factor": toar_scale_factor}})
    
        if bandmath:
            tools.append({'bandmath': {
                "b1" : band_calc,
                "pixel_type": pixel_type
            }})

        if composite:
            tools.append({'composite': {}})
            
        

        #Add pfa tools to order_payload
        order_payload={
            "name": order_name,
            "products": [
                    {
                    "item_ids": item_ids,
                    "item_type": item_type,
                    "product_bundle": prod_bundle
                    }
                ]
        }
        
        #Payload generation w/ tools and alternate delivery methods
        if email:
            order_payload['notifications']={"email": email}
        if webhook:
            order_payload['notifications']={"webhook":{"per_order": webhook,"url": webhook_url}}
            
        if any ((clip, toar, composite)):
            order_payload['tools'] = tools
       
        p(order_payload)
    
        composite_orders.append(order_payload)
    
    return composite_orders

### Submit Composite Orders

In [ ]:
def submit_composite_orders(composite_orders):
    
    order_urls = []
    failed_orders = []
    success_count = 0
    for count, order in enumerate(composite_orders):
        
        response = planet.post(ORDERS_URL, json=order)
        
        if response.status_code == 202:
            order_id = response.json()["id"]
            success_count += 1 
            print ("Order submitted with ID {}".format(order_id))
        else:
            failed_orders.append(order)
            msg = response.json().get("message")
            print ('Order failed with code {} and message "{}"'.format(response.status_code, msg))
            

        order_url = ORDERS_URL + order_id
        print ("Submitted {}".format(order_url))
        order_urls.append(order_url)

        
        sleep(1)
    print ("")
    print ("{} out of {} orders have been successfully submitted".format(success_count, count+1))
    print ("")
    
    return order_urls, failed_orders

### Check Order Response

In [ ]:
def check_response(order_urls):
    
    for order_url in order_urls:

        #Retrieve order state
        response = planet.get(order_url)
        
        order = response.json()
        print ("order state is:", order['state'])
        
        print ("")
        p(order)
        print ("")
        
        while order['state'] == 'running':
            sleep(5)
            # Overwrite existing order with a new response
            order = planet.get(order_url).json()
            print ("order state is:", order['state'])
        
        if order["state"] == "success":
            results_link = order["_links"]["results"]
            order_name = order["name"]
            yield results_link, order_name

# ##########################   Download    ########################## # 

### Download multiple orders (Composite)

In [ ]:
def download_results(order_urls, img_dir):
    
    files = []
    for asset, order_name in check_response(order_urls):
     
        for order_asset in asset:

            name = order_asset["name"]
            print (name)
            fn = os.path.basename(name)
            location = order_asset["location"]
            
            res = planet.get(location, stream=True)
            sleep(1) # Time in seconds. Added this delay and failures to download stopped.

            full_fn = img_dir + order_name + "_" + fn
            if os.path.exists(full_fn):
                print ("{} already exists, skipping".format(full_fn))
                continue
                   
            print ("The filename is: {}".format(full_fn))

            with open(full_fn, "wb") as f:
                for chunk in res.iter_content(chunk_size=1024):
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
                        f.flush()
                    
            files.append(full_fn)

## Main Code 

In [ ]:
#Main 

# Get geometry from json files, build queries, extract ids, and download
for json_file in sorted(glob.glob(json_path+"/*.json")):
    print (json_file)
    
    #Generate Image Directory for results
    dir_name = os.path.splitext(os.path.basename(json_path + json_file))[0]
    img_path = json_path + dir_name + '/'
    print (img_path)
    
    #Create directory if it doesn't already exist
    if not os.path.exists(img_path):
        os.makedirs(img_path)
    
    #Load json file
    with open(json_file) as f:
        json_data = json.load(f)
    
    #Simplify Polygon
    simple_json = simplify_poly(json_data)
    
    #Extract geometry from json file
    (aoi_geometry, clip_geometry) = extract_geometries(simple_json)
    
    #Get Stip Coverage Information
    strips = get_items_with_coverage(planet,
                                 aoi_geometry,
                                 [item_type],
                                 acquired_gte=start_time,
                                 acquired_lte=stop_time,
                                 gsd_gte=min_gsd,
                                 cloud_cover_lte=max_cloud_cover)

    print ("there are a total of %s strips" % len(strips))
    print (" ")
    
    #Check coverage for the strips
    
    thresholded_strips = [s for s in strips if s['coverage'] >= coverage_thresh]

    print ("there are %s strips with at least %0.2f coverage" % (len(thresholded_strips), coverage_thresh))
    print (" ")
    
    #Check images intersect with clip AOI
    (verified_items, removed_items, verified_all) = img_intersection(thresholded_strips,aoi_geometry)
    
    #Group all of the images by their strip IDs
    imgids_by_strips = group_imgs_by_stripid(verified_all,verified_items)
    
    #Create order payloads
    #order_payload = generate_order_payload(dir_name,item_ids,clip_geometry)
    order_payloads = generate_order_payload(dir_name,imgids_by_strips,item_type,prod_bundle,clip,clip_geometry,
                                      toar,toar_scale_factor,bandmath,band_calc,pixel_type,
                                      composite,email,webhook,webhook_url)

    #Save Order_Payload Sample
    order_fn = img_path + 'Order_Payload.json'
    order_payload = order_payloads[0]
    
    with open(order_fn, 'w') as f:
        json.dump(order_payload, f, indent=2)
        
    #Submit composite orders
    #response = planet.post(ORDERS_URL, json=order_payload)
    (order_urls, failed_orders) = submit_composite_orders(order_payloads)    
 
    download_results(order_urls, img_path)